In [ ]:
#| hide
from nbdev.showdoc import *

# core

> Notion Automation for multiple page creation

In [ ]:
#| default_exp core

In [ ]:
#| export
__all__ = ['Database','TriggerDB','JunctionDB','LogDB','AutoLogger']

In [ ]:
#| export 
class Database:
    """Base class for interacting with a Notion database.
    
    Attributes:
        db_id (str): The Notion database ID
        notion: The Notion client instance for API calls
    """
    def __init__(self, db_id, notion):
        """Initialize a Database instance.
        
        Args:
            db_id (str): The Notion database ID
            notion: The Notion client instance
        """
        self.db_id, self.notion = db_id, notion
    
    def get_schema(self):
        """Retrieve the database schema.
        
        Returns:
            dict: Mapping of property names to their types
        """
        db = self.notion.databases.retrieve(self.db_id)
        return {name: props['type'] for name, props in db['properties'].items()}

In [ ]:
#| export
class TriggerDB(Database):
    """Database that triggers logging when updated.
    
    Monitors a database for status changes and extracts related item information
    for downstream logging operations.
    
    Attributes:
        db_id (str): The Notion database ID
        notion: The Notion client instance for API calls
        status_prop (str): Name of the status property to monitor
        relation_prop (str): Name of the relation property linking to related items
        qty_prop (str): Name of the numeric quantity property
    """
    def __init__(self, db_id, notion, status_prop, relation_prop, qty_prop):
        """Initialize a TriggerDB instance.
        
        Args:
            db_id (str): The Notion database ID
            notion: The Notion client instance
            status_prop (str): Name of the status property
            relation_prop (str): Name of the relation property
            qty_prop (str): Name of the quantity property
        """
        super().__init__(db_id, notion)
        self.status_prop, self.relation_prop, self.qty_prop = status_prop, relation_prop, qty_prop
    
    def get_page_data(self, page_id):
        """Extract status, related item ID, and quantity from a page.
        
        Args:
            page_id (str): The Notion page ID to retrieve
            
        Returns:
            tuple: (status, relation_id, qty) where status is str or None,
                   relation_id is str or None, and qty is a number
        """
        page = self.notion.pages.retrieve(page_id)
        status = page['properties'][self.status_prop]['select']['name'] if page['properties'][self.status_prop]['select'] else None
        relation_id = page['properties'][self.relation_prop]['relation'][0]['id'] if page['properties'][self.relation_prop]['relation'] else None
        qty = page['properties'][self.qty_prop]['number']
        return status, relation_id, qty

In [ ]:
#| export
class JunctionDB(Database):
    """Database that connects trigger items to log items with amounts.
    
    Acts as a many-to-many relationship table, storing which items are affected
    by a trigger and their respective amounts or multipliers.
    
    Attributes:
        db_id (str): The Notion database ID
        notion: The Notion client instance for API calls
        relation_prop (str): Name of the property relating to trigger items
        item_prop (str): Name of the property relating to log items
        amount_prop (str): Name of the numeric amount/multiplier property
    """
    def __init__(self, db_id, notion, relation_prop, item_prop, amount_prop):
        """Initialize a JunctionDB instance.
        
        Args:
            db_id (str): The Notion database ID
            notion: The Notion client instance
            relation_prop (str): Name of the relation property to trigger items
            item_prop (str): Name of the relation property to log items
            amount_prop (str): Name of the amount property
        """
        super().__init__(db_id, notion)
        self.relation_prop, self.item_prop, self.amount_prop = relation_prop, item_prop, amount_prop
    
    def get_items(self, relation_id):
        """Get all items and their amounts for a given relation.
        
        Args:
            relation_id (str): The trigger item's relation ID
            
        Returns:
            dict: Mapping of item IDs to their amounts, empty dict if no relation_id
        """
        if not relation_id: return {}
        res = self.notion.data_sources.query(data_source_id=self.db_id, filter={"property": self.relation_prop, "relation": {"contains": relation_id}})
        return {r['properties'][self.item_prop]['relation'][0]['id']: r['properties'][self.amount_prop]['number'] for r in res['results']}

#| export
class LogDB(Database):
    """Database where transaction logs are written.
    
    Records all inventory or quantity changes with item references, amounts,
    triggering events, and reasons for the change.
    
    Attributes:
        db_id (str): The Notion database ID
        notion: The Notion client instance for API calls
        item_prop (str): Name of the property relating to the affected item
        amount_prop (str): Name of the numeric amount property (positive or negative)
        trigger_prop (str): Name of the property relating back to the trigger
        reason_prop (str): Name of the select property describing the reason
    """
    def __init__(self, db_id, notion, item_prop, amount_prop, trigger_prop, reason_prop):
        """Initialize a LogDB instance.
        
        Args:
            db_id (str): The Notion database ID
            notion: The Notion client instance
            item_prop (str): Name of the item relation property
            amount_prop (str): Name of the amount property
            trigger_prop (str): Name of the trigger relation property
            reason_prop (str): Name of the reason select property
        """
        super().__init__(db_id, notion)
        self.item_prop, self.amount_prop, self.trigger_prop, self.reason_prop = item_prop, amount_prop, trigger_prop, reason_prop
    
    def create_entry(self, item_id, amount, trigger_id, reason):
        """Create a new log entry.
        
        Args:
            item_id (str): The ID of the item being logged
            amount (float): The amount to log (positive or negative)
            trigger_id (str): The ID of the triggering page
            reason (str): The reason for this log entry
        """
        self.notion.pages.create(parent={"database_id": self.db_id}, properties={self.item_prop: {"relation": [{"id": item_id}]}, self.amount_prop: {"number": amount}, self.trigger_prop: {"relation": [{"id": trigger_id}]}, self.reason_prop: {"select": {"name": reason}}})

#| export
class AutoLogger:
    """Orchestrates automatic logging from trigger to log via junction.
    
    Monitors trigger database changes and automatically creates corresponding
    log entries by looking up related items through the junction database.
    
    Attributes:
        trigger_db (TriggerDB): The database being monitored for changes
        junction_db (JunctionDB): The database linking triggers to items
        log_db (LogDB): The database where logs are written
        trigger_status (str): The status that activates logging
        multiplier (int): Multiplier for amounts (typically -1 for inventory deduction)
    """
    def __init__(self, trigger_db, junction_db, log_db, trigger_status, multiplier=-1):
        """Initialize an AutoLogger instance.
        
        Args:
            trigger_db (TriggerDB): The trigger database instance
            junction_db (JunctionDB): The junction database instance
            log_db (LogDB): The log database instance
            trigger_status (str): Status value that triggers logging
            multiplier (int, optional): Amount multiplier. Defaults to -1.
        """
        self.trigger_db, self.junction_db, self.log_db = trigger_db, junction_db, log_db
        self.trigger_status, self.multiplier = trigger_status, multiplier
    
    def process(self, page_id):
        """Process a trigger page and create log entries.
        
        Args:
            page_id (str): The trigger page ID to process
            
        Returns:
            str: Status message describing the result
        """
        status, relation_id, qty = self.trigger_db.get_page_data(page_id)
        if status != self.trigger_status: return f"Status is {status}, not {self.trigger_status}"
        items = self.junction_db.get_items(relation_id)
        if not items: return "No items found in junction"
        for item_id, amt in items.items(): self.log_db.create_entry(item_id, self.multiplier * qty * amt, page_id, 'auto_log')
        return f"Logged {len(items)} items"
    
    def cancel(self, page_id):
        """Reverse all logs for a trigger (restore inventory).
        
        Args:
            page_id (str): The trigger page ID to reverse
            
        Returns:
            str: Status message describing the result
        """
        _, relation_id, qty = self.trigger_db.get_page_data(page_id)
        items = self.junction_db.get_items(relation_id)
        if not items: return "No items to reverse"
        for item_id, amt in items.items(): self.log_db.create_entry(item_id, -self.multiplier * qty * amt, page_id, 'cancelled')
        return f"Reversed {len(items)} items"
    
    def adjust_batch(self, page_id, old_qty, new_qty):
        """Log the difference when batch quantity changes.
        
        Args:
            page_id (str): The trigger page ID being adjusted
            old_qty (float): The previous quantity value
            new_qty (float): The new quantity value
            
        Returns:
            str: Status message describing the result
        """
        status, relation_id, _ = self.trigger_db.get_page_data(page_id)
        if status != self.trigger_status: return f"Status is {status}, not active"
        items = self.junction_db.get_items(relation_id)
        if not items: return "No items found"
        delta = new_qty - old_qty
        for item_id, amt in items.items(): self.log_db.create_entry(item_id, self.multiplier * delta * amt, page_id, 'batch_adjusted')
        return f"Adjusted {len(items)} items by {delta} batches"
    
    def handle_update(self, page_id, old_status=None, old_qty=None):
        """Handle any update to trigger page - detects what changed and acts accordingly.
        
        Intelligently determines what type of change occurred (status change, quantity
        adjustment, or activation) and calls the appropriate handler method.
        
        Args:
            page_id (str): The trigger page ID that was updated
            old_status (str, optional): The previous status value. Defaults to None.
            old_qty (float, optional): The previous quantity value. Defaults to None.
            
        Returns:
            str: Status message describing the action taken
        """
        status, _, qty = self.trigger_db.get_page_data(page_id)
        if old_status and old_status == self.trigger_status and status != self.trigger_status: return self.cancel(page_id)
        if status == self.trigger_status:
            if old_status and old_status != self.trigger_status: return self.process(page_id)
            if old_qty and old_qty != qty: return self.adjust_batch(page_id, old_qty, qty)
        return "No action needed"

In [ ]:
show_doc(Database)
show_doc(TriggerDB)
show_doc(JunctionDB)
show_doc(LogDB)


---

[source](https://github.com/amezaikupan/notion_cascade_insert/blob/main/notion_cascade_insert/core.py#L121){target="_blank" style="float:right; font-size:smaller"}

### LogDB

```python

def LogDB(
    db_id, notion, item_prop, amount_prop, trigger_prop, reason_prop
):


```

*Database where transaction logs are written.*

Records all inventory or quantity changes with item references, amounts,
triggering events, and reasons for the change.

Attributes:
    db_id (str): The Notion database ID
    notion: The Notion client instance for API calls
    item_prop (str): Name of the property relating to the affected item
    amount_prop (str): Name of the numeric amount property (positive or negative)
    trigger_prop (str): Name of the property relating back to the trigger
    reason_prop (str): Name of the select property describing the reason

In [ ]:
#|hide 
import nbdev; nbdev.nbdev_export()